In [74]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

In [75]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.applications.vgg16 import VGG16,preprocess_input

In [76]:
from tensorflow.keras.layers import GlobalAveragePooling2D

In [77]:
load_img('/kaggle/input/intel-image-classification/seg_test/seg_test/glacier/23149.jpg')

In [78]:
train_data=ImageDataGenerator(rotation_range=20,horizontal_flip=True,rescale=1.0/255.0,
                             shear_range=0.2,zoom_range=0.2)
train=train_data.flow_from_directory('/kaggle/input/intel-image-classification/seg_train/seg_train',
                                     target_size=(64,64),class_mode='categorical',batch_size=100,
                                     shuffle=True)
test_data=ImageDataGenerator(rescale=1.0/255.0)
test=test_data.flow_from_directory('/kaggle/input/intel-image-classification/seg_test/seg_test',
                                                                    target_size=(64,64),
                                                                    class_mode='categorical',
                                                                    batch_size=100,shuffle=False)

In [79]:
train.class_indices

In [80]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(6,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train,validation_data=test,epochs=10,verbose=2)

In [81]:
df=pd.DataFrame(model.history.history)
df.head()

In [93]:
df[['loss','val_loss']].plot()

In [94]:
df[['accuracy','val_accuracy']].plot()

In [82]:
import cv2
def predict(Image):
    img = cv2.imread(Image)
    img = cv2.resize(img,(64,64))
    img = img/255
    img = img.reshape(-1,64,64,3)
    predict=np.round(model.predict(img)).argmax(axis = 1)[0]
    if predict==0:
        return 'Buildings'
    elif predict==1:
        return 'Forests'
    elif predict==2:
        return 'Glacier'
    elif predict==3:
        return 'Mountains'
    elif predict==4:
        return 'Sea'
    else :
        return 'Streets'

In [83]:
print(predict(('/kaggle/input/intel-image-classification/seg_test/seg_test/forest/23407.jpg')))
print(predict(('/kaggle/input/intel-image-classification/seg_test/seg_test/sea/20513.jpg')))

In [84]:
model2=Sequential()
base=VGG16(input_shape=[64,64,3],weights='imagenet',include_top=False)
base.trainable=True
layers=10
for i in base.layers[0:layers]:
    i.trainable=False
pool=GlobalAveragePooling2D()
first=Dense(100,activation='relu')
second=Dense(6,activation='softmax')
Model=Sequential([base,first,pool,second])
Model.summary()
base.summary()

In [85]:
Model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
Model.fit(train,validation_data=test,epochs=10,verbose=2)

In [86]:
load_img('/kaggle/input/intel-image-classification/seg_test/seg_test/forest/23407.jpg')

In [88]:
import cv2
def Predict(Image):
    img = cv2.imread(Image)
    img = cv2.resize(img,(64,64))
    img = img/255
    img = img.reshape(-1,64,64,3)
    predict=np.round(Model.predict(img)).argmax(axis = 1)[0]
    if predict==0:
        return 'Buildings'
    elif predict==1:
        return 'Forests'
    elif predict==2:
        return 'Glacier'
    elif predict==3:
        return 'Mountains'
    elif predict==4:
        return 'Sea'
    else :
        return 'Streets'

In [89]:
predict('/kaggle/input/intel-image-classification/seg_pred/seg_pred/6234.jpg')

In [90]:
load_img('/kaggle/input/intel-image-classification/seg_pred/seg_pred/6234.jpg')

In [92]:
df1=pd.DataFrame(Model.history.history)
df1.head()

In [95]:
df1[['loss','val_loss']].plot()

In [96]:
df1[['accuracy','val_accuracy']].plot()